In [1]:
%reset
import torch
import torch.nn as nn
from torchvision import models
from collections import defaultdict
import torch.nn.functional as F
from utils.loss import dice_loss
from utils.dataloader import Dataset_3D
import utils.alienlab as alien
from utils.ply import write_ply
import numpy as np
from utils.models import ResNetUNet, my_model_simple
import glob
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import utils.vox_to_coord as vtc

import json
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [2]:
labels_names = ['background', 'flowers', 'peduncle', 'stem', 'leaves', 'fruits']
N_cam = 72
B = 6
path_imgs = "data/arabidopsis/"

Sx = 896 #Center crop
Sy = 448
xinit = 1080 #Original image size
yinit = 1920
label_num = 6
reduction_factor = 18

N_vox = 2197000

In [3]:
def calc_loss(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)

    pred = F.sigmoid(pred)
    dice = dice_loss(pred, target)

    loss = bce * bce_weight + dice * (1 - bce_weight)

    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)

    return loss

def print_metrics(metrics, epoch_samples, phase):
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))

    print("{}: {}".format(phase, ", ".join(outputs)))

    


def train_model(model, optimizer, scheduler, criterium, num_epochs=25):
    xy_full_flat = torch.load('voxel_coord/coordinates_full_pipe.pt').to(device)    
    pred_pad = torch.zeros((N_cam//reduction_factor, label_num, xinit, yinit)).to(device)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])

                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0
            for inputs, labels in dataloaders[phase]:

                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    

                    pred_pad[:,:,(xinit-Sx)//2:(xinit+Sx)//2,
                             (yinit-Sy)//2:(yinit+Sy)//2] = outputs #To fit the camera parameters

                    preds_flat = pred_pad.permute(0,2,3,1)
                    preds_flat = vtc.adjust_predictions(preds_flat)


                    assign_preds = preds_flat[xy_full_flat].reshape(pred_pad.shape[0], 
                                                    xy_full_flat.shape[0]//pred_pad.shape[0], preds_flat.shape[-1])

                    assign_preds[:,:,6] = 0
                    assign_preds = torch.sum(assign_preds, dim = 0)

                    loss = criterium(assign_preds, labels[0])
                    print(loss)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                print("saving best model")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Projection of coordinates

In [4]:
if False:
    
    with open('images.json', 'r') as f:
        pose = json.load(f)

    N_cam = 72
    N_feat = 72

    extrinsics = torch.zeros((N_cam, 3, 4))
    for i in range(N_cam):
        rot = pose[str(i+1)]['rotmat']
        extrinsics[i][:3,:3] = torch.Tensor(rot)
        trans = pose[str(i+1)]['tvec']
        extrinsics[i][:,3] = torch.Tensor(trans)/10.

    with open('cameras.json', 'r') as f:
        focal = json.load(f)
    focal = focal['1']['params']

    r = 1#1080/448

    intrinsics = torch.zeros((1, 3, 3))
    intrinsics[:,0,0] = focal[0]*r
    intrinsics[:,1,1] = focal[0]*r
    intrinsics[:,0,2] = focal[1]*r
    intrinsics[:,1,2] = focal[2]*r
    intrinsics[:,2,2] = 1
    
    cloud_scale = 0.5
    #pred_pad = pred_pad.permute(0,2,3,1)
    #the_shape = pred_pad.shape
    #del pred_pad
    

    N = int(65/cloud_scale)
    #Voxel representation of the point cloud
    min_vec = [int(-40/cloud_scale), int(-40/cloud_scale),int(-5/cloud_scale)] #Limit of the cloud
    basis_voxels = vtc.basis_vox(min_vec, N, N, N)*cloud_scale#List of coordinates 

    #Camera projection
    torch_voxels = torch.from_numpy(basis_voxels)

    #Perspective projection
    xy_coords = vtc.project_coordinates(torch_voxels, intrinsics, extrinsics, give_prod = False)

    #permute x and y coordinates
    xy_coords[:, 2, :] = xy_coords[:,0,:]
    xy_coords[:, 0, :] = xy_coords[:,1,:]
    xy_coords[:, 1, :] = xy_coords[:,2,:]

    coords = vtc.correct_coords_outside(xy_coords, Sx, Sy, xinit, yinit, -1) #correct the coordinates that project outside
    coords = coords[::reduction_factor]
    xy_full_flat = vtc.flatten_coordinates(coords, torch.Size([N_cam//reduction_factor, Sx, Sy, label_num]))
    torch.save(xy_full_flat, 'voxel_coord/coordinates_full_pipe.pt')
    torch.save(torch_voxels, 'voxel_coord/voxels_full_pipe.pt')
    
    del torch_voxels
    del xy_full_flat

In [5]:
def init_3D_set(mode, path = path_imgs, N_subsample = 1, N_cam = N_cam):
    if mode not in ['train', 'val', 'test']:
    
        print("mode should be 'train', 'val' or 'test'")
        
    image_paths = np.sort(glob.glob(path + mode + "/images/*.png"))
    target_paths = np.sort(glob.glob(path + mode + "/3D_label/*.pt"))
    target_paths = np.repeat(target_paths, N_cam)

    return  image_paths[::N_subsample], target_paths[::N_subsample]


trans = transforms.Compose([
    transforms.CenterCrop((896, 448)),
    transforms.ToTensor()])


image_paths, target_paths = init_3D_set('train', N_subsample = reduction_factor)

image_val, target_val = init_3D_set('val', N_subsample = reduction_factor)

image_test, target_test = init_3D_set('test',N_subsample = reduction_factor)

batch_size = 4

train_dataset = Dataset_3D(image_paths, target_paths, transform=trans)
val_dataset = Dataset_3D(image_val, target_val, transform=trans)
test_dataset = Dataset_3D(image_test, target_test, transform=trans)


image_datasets = {
        'train': train_dataset, 'val': val_dataset , 'test': test_dataset
        }

dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0),
    'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0),
    'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    }

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

num_class = 6
model = ResNetUNet(num_class).to(device)

# freeze backbone layers
for l in model.base_layers:
    for param in l.parameters():
        param.requires_grad = False

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=30, gamma=0.1)

xy_full_flat = torch.load('voxel_coord/coordinates_05_shift6.pt')
xy_full_flat = xy_full_flat.reshape(N_vox, N_cam)
xy_full_flat = xy_full_flat[:,::reduction_factor]


wb = 5/1e6
wc = 1/1e4
weights = [wb, wc, wc, wc, wc, wc, wb] #[ 1 / number of instances for each class]
class_weights = torch.FloatTensor(weights).cuda()

#criterium = nn.NLLLoss(weights = class_weights)
criterium = nn.CrossEntropyLoss(weight=class_weights)

model = train_model(model, optimizer_ft, exp_lr_scheduler, criterium, num_epochs=5)
torch.save(model, 'model_weights/attempt_model.pt')

cuda:0
Epoch 0/4
----------
LR 0.0001
tensor(1.9419, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.9327, device='cuda:0', grad_fn=<NllLossBackward>)


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

### Fully trained pipe

In [7]:
reduction_factor = 1
image_paths, target_paths = init_3D_set('train', N_subsample = reduction_factor)

batch_size = 4
N_views = 72
n = 9

train_dataset = Dataset_3D(image_paths[N_views*n:N_views*(n+1)], target_paths[N_views*n:N_views*(n+1)], 
                           transform=trans)

loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

model_segmentation = torch.load('model_weights/unet_448_long_train.pt') #needs utils.models imported
model_classification = torch.load("classification_trials/shifting_blur1_weight_classview_bias_none_lr_005_wb_1e5_wc_1e4.pt")
with torch.no_grad():


    pred_tot = []
    for inputs, labels in loader:
        inputs = inputs.to(device)
        outputs = model_segmentation(inputs)
        pred_tot.append(outputs)

    pred_tot = torch.cat(pred_tot, dim = 0) #All predictions into one tensor
    pred_tot_class = pred_tot[:,1:,:,:]
    #pred_tot_class = nn.MaxPool2d(sf*2)(pred_tot_class)
    #pred_tot_class = nn.Upsample(scale_factor = sf*2)(pred_tot_class)
    pred_tot[:,1:,:,:] = pred_tot_class

    pred_pad = torch.zeros((N_cam, label_num, xinit, yinit))
    Sbix = pred_tot.shape[2]
    Sbiy = pred_tot.shape[3]
    pred_pad[:,:,(xinit-Sbix)//2:(xinit+Sbix)//2,(yinit-Sbiy)//2:(yinit+Sbiy)//2] = pred_tot #To fit the camera parameters
    del pred_tot
    pred_pad = pred_pad.permute(0,2,3,1)

    preds_flat = vtc.adjust_predictions(pred_pad)
    
    total_points = []
    for i in range(8):
        xy_full_flat = torch.load('voxel_coord/coordinates_05_shift%d.pt'%i).to(device)
        assign_preds = preds_flat[xy_full_flat].reshape(pred_pad.shape[0], 
                                                        xy_full_flat.shape[0]//pred_pad.shape[0], preds_flat.shape[-1])
        del xy_full_flat
        #assign_preds = assign_preds[:,:,:-1]
        #assign_preds = torch.sum(assign_preds, dim =  0)
        assign_preds = assign_preds.permute(1, 0, 2).unsqueeze(0)
        assign_preds = model_classification[0](assign_preds.to(device))
        assign_preds = torch.argmax(assign_preds[0, :, :-1], dim = 1)
        vol = torch.load('voxel_coord/voxels_05_shift%d.pt'%i)
        ind = (assign_preds != 0)*(assign_preds != 4)
        vol = vol[ind]
        vol[:,3] = assign_preds[ind]
        total_points.append(vol)
        del vol
        

torch.Size([72, 2197000, 7])
torch.Size([1, 2197000, 72, 7]) torch.Size([1, 1, 72, 7])
torch.Size([1, 2197000, 72, 7])
torch.Size([1, 2197000, 1, 7])
torch.Size([3773, 4])
torch.Size([72, 2197000, 7])
torch.Size([1, 2197000, 72, 7]) torch.Size([1, 1, 72, 7])
torch.Size([1, 2197000, 72, 7])
torch.Size([1, 2197000, 1, 7])
torch.Size([3765, 4])
torch.Size([72, 2197000, 7])
torch.Size([1, 2197000, 72, 7]) torch.Size([1, 1, 72, 7])
torch.Size([1, 2197000, 72, 7])
torch.Size([1, 2197000, 1, 7])
torch.Size([3714, 4])
torch.Size([72, 2197000, 7])
torch.Size([1, 2197000, 72, 7]) torch.Size([1, 1, 72, 7])
torch.Size([1, 2197000, 72, 7])
torch.Size([1, 2197000, 1, 7])
torch.Size([3771, 4])
torch.Size([72, 2197000, 7])
torch.Size([1, 2197000, 72, 7]) torch.Size([1, 1, 72, 7])
torch.Size([1, 2197000, 72, 7])
torch.Size([1, 2197000, 1, 7])
torch.Size([3761, 4])
torch.Size([72, 2197000, 7])
torch.Size([1, 2197000, 72, 7]) torch.Size([1, 1, 72, 7])
torch.Size([1, 2197000, 72, 7])
torch.Size([1, 219700

In [8]:
total_points = torch.cat(total_points, dim = 0)

In [9]:
write_ply('high_quality_class.ply', total_points.detach().cpu().numpy(),
      ['x', 'y', 'z', 'labels'])

True

In [10]:
assign_preds = torch.argmax(assign_preds, dim = 1)assign_preds.shape

torch.Size([2197000])

In [11]:
ind.shape

torch.Size([2197000])

In [12]:
vol.shape

NameError: name 'vol' is not defined

In [ ]:
assign_preds.shape